In [2]:
import pandas as pd
import numpy as np
import sqlite3
import ast
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
def find_similar_n(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [3]:
user_response = pd.read_csv('/Users/tristannisbet/Documents/SM/survey_responses.csv')

In [5]:
user_response.rename(columns = {'What country are you from? ': 'nationality', 'Age Range': 'age', 'Gender': 'gender',
                    '1. Choose your top favorite 3-5 cities you have traveled to that are on this list.  - Favorite City #1': 'favorite_city_one',
                    '2. Favorite city #2': 'favorite_city_two', '3. Favorite city #3': 'favorite_city_three',
                    '4. Favorite city #4': 'favorite_city_four', '5. Favorite city #5': 'favorite_city_five',
                    "6. If there's a city you have been and loved that is not on this list, add it below. ": 'extra_favorite',
                    "7. What cities on this list have you been to and not enjoyed?   - Least favorite city #1": 'least_favorite_one',
                    '8. Least favorite city #2': 'least_favorite_two', 
                    "9. If there's a city you have been to and haven't liked that is not on this list, add it below": 'extra_least_favorite',
                    "What price range of restaurant do you eat at when you travel? [Price level: 1 (Fast/Cheap Eats)]": 'food_one',
                    "What price range of restaurant do you eat at when you travel? [Price level: 2 (Casual Dining)]": 'food_two',
                    "What price range of restaurant do you eat at when you travel? [Price level: 3 (Upscale Dining)]": 'food_three',
                    "What price range of restaurant do you eat at when you travel? [Price level: 4 (Fine Dining/High End)]": 'food_four',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Art Gallery]": 'art_gallery',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Library]": 'library',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Museum ]": 'museum',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Aquarium]": 'aquarium',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Amusement Park ]": 'amusement_park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Zoo]": 'zoo',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Movie Theater]": 'movie_theater',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Mall / Souvenir shop ]": 'store',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Park ]": 'park',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Natural Feature / Beach]": 'natural_feature',
                    "From the following types of tourist attractions, which ones are you likely to go do while traveling? [Place of Worship (Church/Temple)]": 'place_of_worship'}, inplace=True )

In [6]:
user_response.fillna('None', inplace=True)

In [7]:
user_response.drop(columns=['Timestamp', 'extra_favorite', 'extra_least_favorite'], inplace=True)

In [ ]:
user_city_only = user_response.copy()
user_city_only.drop(columns=['nationality', 'age', 'gender'], inplace=True)
user_city_only

In [ ]:
dummy = pd.get_dummies(df)
dummy_city = pd.get_dummies(user_city_only)

In [ ]:
cosine_user = cosine_similarity(dummy)
cosine = cosine_similarity(dummy_city)

In [ ]:
# cosine does not have user demographics 
cosine_user = pd.DataFrame(cosine_user)
cosine = pd.DataFrame(cosine)

In [ ]:
cosine_user

In [ ]:
cosine

In [ ]:
top_10_user = find_similar_n(cosine_user,10)
top_10_user_c= find_similar_n(cosine,10)

In [ ]:
top_10_user

In [ ]:
top_10_user_c

## Building city matrix


In [18]:
one = get_df('restaurants_one')
two = get_df('restaurants_two')
three = get_df('restaurants_three')
four = get_df('restaurants_four')
top_rest = get_df('restaurants')
all_food = pd.concat([one, two, three, four, top_rest], axis =0)
all_food.sample(5)

,country,city,name,address,price_level,rating,user_ratings_total,types,latitude,longitude,place_id,id
5100,United Arab Emirates,Abu Dhabi,El Sombrero,Sheraton Abu Dhabi Hotel & Resort، Corniche Ro...,2.0,4.1,183,"['restaurant', 'food', 'point_of_interest', 'e...",24.500194,54.369081,ChIJ2URIyIpmXj4Rp2gL9h3zBjM,94
587,India,Mumbai,Mamagoto,"Gazebo House, 133, Ground, Hill Rd, Bandra Wes...",3.0,4.2,1708,"['restaurant', 'food', 'point_of_interest', 'e...",19.054929,72.828417,ChIJ76H1gz_J5zsR-vq1_venRZk,14
1217,China,Guangzhou,Tiger Prawn Vietnamese Restaurant,"China, Guangdong Province, Guangzhou, Yuexiu D...",NaN,4.6,32,"['restaurant', 'food', 'point_of_interest', 'e...",23.122793,113.267298,ChIJK4o9k8D4AjQRy8ogacy6Oe8,21
560,Czech Republic,Prague,Restaurace Bellevue,"Karoliny Světlé 34, 110 00 Staré Město, Czechia",4.0,4.7,1105,"['restaurant', 'food', 'point_of_interest', 'e...",50.084747,14.414117,ChIJC7EqeeWUC0cRQXjyLpVKugE,22
5891,Germany,Düsseldorf,Tonhalle Kaiserswerth,"Klemenspl. 7, 40489 Düsseldorf, Germany",2.0,4.2,570,"['restaurant', 'bar', 'food', 'point_of_intere...",51.301038,6.739888,ChIJ57EHb5W3uEcR4JP21I_syvI,108


In [20]:
attractions = get_df('attractions')
attractions.sample(5)

,country,city,name,address,price_level,rating,user_ratings_total,types,latitude,longitude,place_id,id
1329,Saudi Arabia,Medina,Masjid musabbih,"7697 الضحاك المشرقى, Al Usbah, 3337، Medina 42...",None,4.7,168.0,"['point_of_interest', 'establishment']",24.435846,39.615380,ChIJT9nmVYG_vRURZoPjG6hD-oA,23
3693,Portugal,Lisbon,Carmo Convent,"Largo do Carmo, 1200-092 Lisboa, Portugal",None,4.5,8987.0,"['tourist_attraction', 'museum', 'point_of_int...",38.712141,-9.140246,ChIJP8YuN380GQ0R7yzgkmX11zI,63
5166,Germany,Frankfurt am Main,Alte Brücke,"Alte Brücke, 60594 Frankfurt am Main, Germany",None,4.7,581.0,"['tourist_attraction', 'point_of_interest', 'e...",50.107548,8.687809,ChIJ28th2KAOvUcRPxrtGPeIf3k,87
1108,Taiwan,Taipei,National Museum of History,"No. 49號, Nanhai Road, Zhongzheng District, Tai...",None,4.2,2276.0,"['museum', 'tourist_attraction', 'point_of_int...",25.031570,121.511199,ChIJzx49raOpQjQR7NYRaIqDppk,19
4615,New Zealand,Auckland,Domain Wintergardens,"Wintergarden Road, Parnell, Auckland 1010, New...",None,4.6,2094.0,"['tourist_attraction', 'park', 'point_of_inter...",-36.860240,174.774093,ChIJCQ6TugpIDW0RU_XfsaBVC0E,78


In [21]:
def to_city(df):
    city_df = df.groupby(['country', 'city', 'id', 'price_level'])['name'].count().to_frame()
    price_level = city_df.pivot_table(index=['country', 'city', 'id'], columns='price_level', values='name', aggfunc='first')
    price_level['avg_price'] = df.groupby(['country', 'city', 'id'])['price_level'].mean()

    
    return price_level

In [22]:
food = to_city(all_food)
food

price_level                      1.0   2.0   3.0   4.0  avg_price
country   city             id                                    
Argentina Buenos Aires     85    6.0  80.0  68.0  31.0   2.670270
Australia Melbourne        73   18.0  84.0  79.0  19.0   2.495000
          Sydney           55   26.0  14.0  67.0  29.0   2.727941
Austria   Vienna           37   65.0  86.0  78.0  19.0   2.205645
Belgium   Brussels         60   10.0  89.0  44.0  18.0   2.434783
...                              ...   ...   ...   ...        ...
Uruguay   Montevideo       129   7.0  93.0  27.0   5.0   2.227273
Vietnam   Da Nang          90   18.0  78.0   8.0   NaN   1.903846
          Ha Long          48   15.0  23.0   1.0   NaN   1.641026
          Hanoi            52   22.0  81.0  15.0   1.0   1.957983
          Ho Chi Minh City 31   18.0  84.0  68.0   4.0   2.333333

[136 rows x 5 columns]

In [23]:
place_of_worship = ['place_of_worship', 'hindu_temple', 'church', 'mosque', 'synagogue']
shopping = ['store', 'shopping_mall', 'clothing_store', 'electronics_store', 'grocery_or_supermarket', 'department_store']

attractions_to_keep = ['amusement_park', 'museum', 'park', 'art_gallery', 'aquarium',
                      'zoo', 'library', 'movie_theater', 'natural_feature'] + place_of_worship + shopping

In [24]:
def main_call(attraction_df):
    attractions_split = split_types(attraction_df)
    dummy = dummies(attractions_split)
    by_city, all_attractions = attraction_count(dummy, attractions_split)
    
    return by_city, all_attractions

def split_types(df):
    df['split_types'] = [ast.literal_eval(x) for x in df.types]
    df['split_types_str'] = [','.join(x) for x in df.split_types]
    
    return df

def dummies(df):
    dummies = df.split_types_str.str.get_dummies(sep=',')

    return dummies

def attraction_count(dummies_df, all_attractions_df):
    all_attractions_df = pd.concat([all_attractions_df, dummies_df], axis=1)
    type_col_names = attractions_to_keep
    type_col_names.extend(['country', 'city', 'id'])
    attraction_count = all_attractions_df[type_col_names].groupby(['country', 'city', 'id']).sum()
    
    return attraction_count, all_attractions_df

In [25]:
city_group, all_attractions2 = main_call(attractions2)

In [ ]:
all_attractions

In [26]:
city_group

amusement_park  museum  park  art_gallery  \
country   city             id                                               
Argentina Buenos Aires     85                1      16    17            1   
Australia Melbourne        73                2      10    14            5   
          Sydney           55                1       8    15            2   
Austria   Vienna           37                1      25     7            0   
Belgium   Brussels         60                2      20     9            0   
...                                        ...     ...   ...          ...   
Uruguay   Montevideo       129               1      18     9            0   
Vietnam   Da Nang          90                1       7     5            0   
          Ha Long          48                1       2     1            0   
          Hanoi            52                0      11     6            0   
          Ho Chi Minh City 31                5      10    10            2   

                                aquarium  zoo  library  movie_theater  \
country   city             id                                           
Argentina Buenos Aires     85          0    1        0              0   
Australia Melbourne        73          1    2        1              1   
          Sydney           55          1    2        0              0   
Austria   Vienna           37          1    3        1              0   
Belgium   Brussels         60          0    0        1              0   
...                                  ...  ...      ...            ...   
Uruguay   Montevideo       129         0    0        1              0   
Vietnam   Da Nang          90          0    0        0              0   
          Ha Long          48          0    0        0              0   
          Hanoi            52          0    1        0              0   
          Ho Chi Minh City 31          0    1        0              0   

                                natural_feature  place_of_worship  \
country   city             id                                       
Argentina Buenos Aires     85                 0                 1   
Australia Melbourne        73                 0                 2   
          Sydney           55                 1                 1   
Austria   Vienna           37                 0                 3   
Belgium   Brussels         60                 0                 4   
...                                         ...               ...   
Uruguay   Montevideo       129                0                 1   
Vietnam   Da Nang          90                 0                11   
          Ha Long          48                 1                 0   
          Hanoi            52                 0                 7   
          Ho Chi Minh City 31                 0                 8   

                                hindu_temple  church  mosque  synagogue  \
country   city             id                                             
Argentina Buenos Aires     85              0       1       0          0   
Australia Melbourne        73              0       2       0          0   
          Sydney           55              0       1       0          0   
Austria   Vienna           37              0       3       0          0   
Belgium   Brussels         60              0       4       0          0   
...                                      ...     ...     ...        ...   
Uruguay   Montevideo       129             0       1       0          0   
Vietnam   Da Nang          90              0       3       0          0   
          Ha Long          48              0       0       0          0   
          Hanoi            52              0       2       0          0   
          Ho Chi Minh City 31              0       1       0          0   

                                store  shopping_mall  clothing_store  \
country   city             id                                          
Argentina Buenos Aires     85       0              1          

In [27]:
def attraction_food(attraction_df, food_df):
    city = pd.merge(attraction_df, food_df, on = ['country', 'city', 'id'], how = 'outer')
    city.reset_index(inplace=True)
    city['id'] = pd.to_numeric(city['id'])
    city.set_index(['country', 'city', 'id'], inplace=True)
    city.sort_values('id', inplace=True)
    return city

In [28]:
city = attraction_food(food, city_group)

In [29]:
city

,,,1.0,2.0,3.0,4.0,avg_price,amusement_park,museum,park,art_gallery,aquarium,...,hindu_temple,church,mosque,synagogue,store,shopping_mall,clothing_store,electronics_store,grocery_or_supermarket,department_store
country,city,id,,,,,,,,,,,,,,,,,,,,,
Hong Kong,Hong Kong,1,62.0,68.0,70.0,86.0,2.629371,4,7,5,1,0,...,0,1,0,0,0,0,0,0,0,0
Thailand,Bangkok,2,62.0,72.0,70.0,39.0,2.353909,1,13,10,2,1,...,0,2,0,0,1,0,0,0,0,0
United Kingdom,London,3,62.0,85.0,77.0,73.0,2.542088,0,19,8,4,0,...,0,2,0,0,4,0,0,0,0,0
Macau,Macau,4,NaN,44.0,12.0,4.0,2.333333,2,10,6,1,0,...,0,6,0,0,0,0,0,0,0,0
Singapore,Singapore,5,11.0,63.0,80.0,60.0,2.883178,3,7,16,0,1,...,1,0,1,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ghana,Accra,135,9.0,94.0,18.0,4.0,2.136000,0,4,1,1,0,...,0,9,1,0,1,1,0,0,0,0
Ecuador,Quito,136,14.0,82.0,37.0,3.0,2.213235,2,13,18,1,0,...,0,7,0,0,0,0,0,0,0,0
China,Tianjin,137,1.0,13.0,5.0,2.0,2.380952,1,10,13,0,1,...,0,2,0,0,0,0,0,0,0,0


In [ ]:
# Filling null values so cosine will work
city[city[1.0].isnull()]
city.fillna(0, inplace=True)

In [ ]:
cosine_city = pd.DataFrame(cosine_similarity(city))

cosine_city

In [ ]:
top_10_city = find_similar_n(cosine_city, 10)
top_10_city

### All user and city matrix completed

In [ ]:
def select_favorite():
    

In [10]:
user_response.iloc[3]['favorite_city_one']

'Rome'

In [32]:
k = 0 
while k < 3:
    for user in user_response.iloc[k]:
        two = user_response.iloc[k]['favorite_city_two']
        three = user_response.iloc[k]['favorite_city_three']
        one =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_one']]
        two =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_two']]
        three =  city.iloc[city.index.get_level_values('city') == user_response.iloc[k]['favorite_city_three']]
        top = pd.concat([one, two, three])
    print(top)
    print(k)
    k += 1

                       1.0   2.0   3.0   4.0  avg_price  amusement_park  \
country  city     id                                                      
Thailand Krabi    99  27.0  27.0   1.0   NaN   1.527273               1   
Italy    Rome     16  69.0  89.0  70.0  24.0   2.194444               0   
Hungary  Budapest 62  14.0  88.0  70.0  19.0   2.492147               0   

                      museum  park  art_gallery  aquarium  ...  hindu_temple  \
country  city     id                                       ...                 
Thailand Krabi    99       3     8            0         0  ...             0   
Italy    Rome     16      17     7            2         0  ...             0   
Hungary  Budapest 62      10    12            1         1  ...             0   

                      church  mosque  synagogue  store  shopping_mall  \
country  city     id                                                    
Thailand Krabi    99       0       0          0      0              0   
Ital